# 웹 데이터 수집 및 시각화

## 1. 웹 크롤링으로 데이터 수집

### #그래프에서 한글사용하는 방법
- **(코랩에서)한글폰트 설치하기**

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 코랩에서 위 코드를 실행시킨 후  반드시 코랩 메뉴: "런타임>런타임 다시 시작" 합니다.

- **한글 폰트 지정하기**

In [1]:
# 코랩에서 한글 폰트 종류와 이름이 win과 다를 수 있다!!!
# 코랩: NanumGothic, 윈도우: Malgun Gothic
import matplotlib.pyplot as plt
plt.rcParams.update({'font.family': 'NanumGothic',
                     'font.size': 12,
                     'figure.figsize': (6, 4),
                     'axes.unicode_minus':  False }) # 폰트 설정

### 1) 웹 크롤링 기초

### 라이브러리 설치하기

In [ ]:
# 정적 크롤링을 위한 requests 설치
!pip install requests

In [ ]:
# HTML과 XML 문서를 파싱하기 위한 파이썬 패키지
!pip install beautifulsoup4

In [ ]:
# 동적 크롤링을 위한 셀레니움 설치
!pip install selenium

In [ ]:
# 동적 크롤링을 위한 크롬드라이버 자동설치 라이브러리 설치
!pip install chromedriver-autoinstaller

In [ ]:
import requests
import bs4
import selenium
print(requests.__version__)
print(bs4.__version__)
print(selenium.__version__)

### 1) 웹 크롤링 이해하기

- 웹 페이지(html) 가져오기

In [ ]:
import requests

# 웹 페이지 요청하기(결과 Response=200 이면 성공!)
requests.get("https://www.naver.com")

In [ ]:
import requests

response = requests.get("https://google.com")

# 응답 상태
print('#응답 상태: ', response.status_code)

# 응답 바이너리 원문
print('#응답 바이너리 원문: ', response.content)

# 응답 UTF-8로 인코딩된 문자열
print('#응답 UTF-8로 인코딩된 문자열: ', response.text)

# 응답 헤더
print('#응답 헤더: ', response.headers)

# 응답 헤더: 콘텐트 유형
print('#응답 헤더유형: ', response.headers['Content-Type'])



---



### 2) 정적 크롤링(스크래핑)

#### **[실습] : 할리스 커피매장 정보 수집하기**
 - 매장정보 : https://www.hollys.co.kr/store/korea/korStore.do
 - 검색조건 사용 링크 : https://www.hollys.co.kr/store/korea/korStore2.do?pageNo=2&sido=&gugun=&store=


In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import datetime

MAX = 100    # 총 가져올 건수
FILE = './hollys_매장정보.csv'

#[CODE 1]
def hollys_store(result):
    cnt, MAX_flag = 0, False
    for page in range(1,59):
        Hollys_url = 'https://www.hollys.co.kr/store/korea/korStore2.do?pageNo=%d&sido=&gugun=&store=' %page
        print(Hollys_url)
        html = urllib.request.urlopen(Hollys_url)
        # 데이터 파싱하기
        soupHollys = BeautifulSoup(html, 'html.parser')
        tag_tbody = soupHollys.find('tbody')
        for store in tag_tbody.find_all('tr'):
            if len(store) <= 3:
                break
            store_td = store.find_all('td')
            store_name = store_td[1].string
            store_sido = store_td[0].string
            store_address = store_td[3].string
            store_phone = store_td[5].string
            result.append([store_name]+[store_sido]+[store_address]
                          +[store_phone])
            cnt += 1
            print(f'[{cnt:0>3}] {store_name}  \t {store_sido}')
            if cnt == MAX:
                MAX_flag = True
                break
        if MAX_flag:
            break

    return

#[CODE 0]
def main():
    result = []
    print('Hollys store crawling >>>>>>>>>>>>>>>>>>>>>>>>>>')
    hollys_store(result)   #[CODE 1] 호출
    hollys_tbl = pd.DataFrame(result, columns=('store', 'sido-gu', 'address','phone'))
    hollys_tbl.to_csv(FILE, encoding='cp949', mode='w', index=True)
    del result[:]
    return hollys_tbl


df = main()
df

#### **[실습] :  텍스트 데이터를 토큰화하여 빈도수를 구하고 빈도수에 따른 워드클라우드 만들기**
- 앞에서 수집한 할리스 커피매장의 매장 위치 정보 중 지역(**sido-gu**) 정보를 이용하여 워드 클라우드 만들기
- 필요한 라이브러리 및 주의사항
    - 사이킷런 CountVectorizer클래스 사용 : 단어 빈도수 추출
    - 단, 한글에서 불용어 처리 및 가중치 처리 등 자연어 텍스트 전처리에 필요한 여러 가지 방법은 여기서 다루지 않는다.

In [ ]:
# 사이킷런 설치하기
#!pip install scikit-learn

In [ ]:
#!pip install wordcloud

In [ ]:
import sklearn
sklearn.__version__

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10,6)

# 말뭉치를 토큰화하여 빈도수 가져오기
def get_wordTokenCount(corpus):
    from sklearn.feature_extraction.text import CountVectorizer

    # 말뭉치를 토큰화하기
    vect = CountVectorizer().fit(corpus)
    count = vect.transform(corpus).toarray().sum(axis=0)

    # 토큰 빈도수로 정렬하고 토큰명 추출
    idx = np.argsort(-count)  # 내림 정렬하여 인덱스 반환: 토큰의 인덱스
    count = count[idx]        # 토큰의 빈도수
    feature_name = np.array(vect.get_feature_names_out())[idx]  # 토큰값

    # 빈도수 많은 순서대로 토큰명 10개만 출력
    print(list(zip(feature_name, count))[:10])

    return feature_name, count


# 단어(토큰) 빈도수 막대 그래프 그리기
def draw_wordTokenCountGraph(data, freq):
    plt.bar(data, freq)
    plt.grid()
    plt.show()

    # 그래프 그림 저장히기
    plt.savefig(f'./token_bar_graph.png')

# 워드클라우드 만들기
def make_wordcloud(feature_name, count):
    # 한글 폰트 경로를 설정
    # font_path = 'malgun'  # C:/Windows/Fonts/    # window폰트
    font_path = 'NanumGothic'  #/usr/share/fonts/truetype/nanum/NanumGothic.ttf  #코랩

    # (토큰명, 빈도수) 딕셔너리 타입으로 변환
    data = dict(zip(feature_name, count))

    # 워드클라우드로 그래프로 시각화
    wc = WordCloud(width = 1000, height = 600, background_color="white", font_path=font_path)
    plt.imshow(wc.generate_from_frequencies(data)) #딕셔너리
    plt.axis("off")
    plt.show()

    # 이미지 파일로 저장하기
    wc.to_file(f'./워드클라우드.png')


# 1.텍스트 말뭉치(corpus) 데이터 지정하기
corpus = df['sido-gu'].to_list()
# print(corpus)

# 2.말뭉치를 토큰화하여 빈도수 가져오기
feature_name, count = get_wordTokenCount(corpus)

# 3.단어(토큰) 빈도수 막대 그래프 그리기(상위 10개)
# draw_wordTokenCountGraph(feature_name[:10], count[:10])

# 4.워드 클라우드 만들기
make_wordcloud(feature_name, count)



---

